### 1. 为什么使用isinstance而不使用type？
* type() 不会认为子类是一种父类类型，不考虑继承关系。
* isinstance() 会认为子类是一种父类类型，考虑继承关系。

In [5]:
class my_list(list):
    pass

a = my_list()
print(type(a))
print(type([]))
isinstance(a,list)

<class '__main__.my_list'>
<class 'list'>


True

### 2. 使用except去捕获异常

In [7]:
try:
    1/0
except ZeroDivisionError:
    print('THE 0 is error')

THE 0 is error


### 3. 使用装饰器

#### 1.装饰器基本
为什么编写装饰器要用到闭包？因为装饰器函数返回的是一个函数，而如果要在输入函数前后都能插入语句，那必须再创建一个函数。
如果不返回一个函数，直接执行输入函数功能，则是一个回调函数了。

In [1]:
import time
def count_time(func):
    def wrapper():
        s = time.time()
        res = func()
        e = time.time()
        print(e-s)
        return res
    return wrapper

@count_time
def test():
    for i in range(1000000):
        a = 1
        pass
    return 1

# count_time(test)()
a = test()
print(a)

0.022235870361328125
1


#### 2.使用带参数装饰器
上面代码，如果要对有参数的func进行计时怎么办？
在闭包函数内，增加需要传入的参数

In [17]:
import time
def count_time(func):
    def wrapper(*args,**kwargs):
        s = time.time()
        res = func(*args,**kwargs)
        e = time.time()
        print(e-s)
        return res
    return wrapper

@count_time
def test(n):
    if n==0:
        return 1
    return n*test(n-1)

def test2(n):
    if n==0:
        return 1
    return n*test2(n-1)

a = count_time(test2)
a(18)
# test(18)

0.0


6402373705728000

根据上面可以知道，如果直接对递归元素使用装饰器，则会导致循环调用装饰器。原因应该是使用装饰器等价于
```python
test2 = count_time(test2)
test2(18)
```
导致原函数递归变为装饰器函数递归

#### 3.带参数的装饰器

In [39]:
from functools import wraps

# 将参数和func参数放一起
def count_time(func,thr):
    def wrapper(*args,**kwargs):
        s = time.time()
        res = func(*args,**kwargs)
        e = time.time()
        print(e-s)
        if thr>2:
            raise TimeoutError
        return res
    return wrapper

def test2(n):
    for i in range(n):
        time.sleep(0.1)

count_time(test2,2)(3)  #手工调用


# 但是对于装饰器传参，根据装饰器规则定义需要在外层再包一层函数

def count_time_dec(thr):
    def inner(func):
        @wraps(func)   #防止重写func函数名称
        def wrapper(*args,**kwargs):
            s = time.time()
            res = func(*args,**kwargs)
            e = time.time()
            print(e-s)
            if e-s>thr:
                raise TimeoutError
            return res
        return wrapper
    return inner

@count_time_dec(thr=1)
def test3(n):
    for i in range(n):
        time.sleep(0.1)

count_time_dec(3)(test3)(3)
test3(3)
print(test3.__name__)

0.3200678825378418
0.3111133575439453
0.3111133575439453
0.33655810356140137
test3


#### 4.装饰器类
为了更好的复用和继承，可以使用装饰器类

In [44]:
from functools import wraps
 
class logit(object):
    def __init__(self, logfile='out.log'):
        self.logfile = logfile
 
    def __call__(self, func):
        @wraps(func)
        def wrapped_function(*args, **kwargs):
            log_string = func.__name__ + " was called"
            print(log_string)
            # 打开logfile并写入
            with open(self.logfile, 'a') as opened_file:
                # 现在将日志打到指定的文件
                opened_file.write(log_string + '\n')
            # 现在，发送一个通知
            self.notify()
            return func(*args, **kwargs)
        return wrapped_function
 
    def notify(self):
        # logit只打日志，不做别的
        print('正在写入日志')

In [41]:
class email_logit(logit):
    '''
    一个logit的实现版本，可以在函数调用时发送email给管理员
    '''
    def __init__(self, email='admin@myproject.com', *args, **kwargs):
        self.email = email
        super(email_logit, self).__init__(*args, **kwargs)
 
    def notify(self):
        # 发送一封email到self.email
        # 这里就不做实现了
        pass

In [46]:
@logit()
def myfunc1():
    pass
myfunc1()

myfunc1 was called
正在写入日志


## 生成器

In [ ]:
s = (i for i in range(100000000000))  #生成器,惰性求值

In [6]:
m = [i for i in range(10000000)]

In [22]:
def get_val():      # 调用函数返回一个生成器，迭代生成器
    yield 1
    yield 2
    yield 3

def my(n):
    i = 0
    while i <n:
        yield i
        i+=1
for i in my(19):
    print(i) 

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
